In [1]:
import re
import string
import pandas as pd
import numpy as np
from collections import Counter
import sqlite3
import json
import csv

import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

# import fasttext
# import gensim
# from gensim.models import Word2Vec
# from gensim.models import ldaseqmodel
# from gensim import corpora
# import gensim.downloader as api

from lxml import etree

import matplotlib.pyplot as plt


from time import time
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

from tqdm import tqdm

plt.rcParams['figure.dpi'] = 300
plt.rcParams["font.family"] = "serif"


def clean_text(text, tokenizer, stopwords):
    """Pre-process text and generate tokens

    Args:
        text: Text to tokenize.

    Returns:
        Tokenized text.
    """
    text = str(text).lower()  # Lowercase words
    text = re.sub(r"\[(.*?)\]", "", text)  # Remove [+XYZ chars] in content
    text = re.sub(r"\s+", " ", text)  # Remove multiple spaces in content
    text = re.sub(r"\w+…|…", "", text)  # Remove ellipsis (and last word)
    text = re.sub(r"(?<=\w)-(?=\w)", " ", text)  # Replace dash between words
    text = re.sub(
        f"[{re.escape(string.punctuation)}]", "", text
    )  # Remove punctuation

    tokens = tokenizer(text)  # Get tokens from text
    tokens = [t for t in tokens if not t in stopwords]  # Remove stopwords
    tokens = ["" if t.isdigit() else t for t in tokens]  # Remove digits
    tokens = [t for t in tokens if len(t) > 1]  # Remove short tokens
    
    # stemming and lem
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(w) for w in tokens]
    
    return tokens

In [4]:
path = './s2orc_hci/pdf_parses/pdf_parses_0.jsonl'

# load jsonl file
with open(path, 'r', encoding='utf8') as f:
    data = f.readlines()
    data = [json.loads(line) for line in data]

# for dSec in data[1]['body_text']:
#     text = dSec['section'].lower()
#     regexp = re.compile(r'introduction')
#     if regexp.search(text):
#         print(dSec)

# for dSec in data[1]['body_text']:
#     text = dSec['section'].lower()
#     regexp = re.compile(r'related work')
#     if regexp.search(text):
#         print(dSec)


for dSec in data[3]['body_text']:
    text = str(dSec).lower()
    print(dSec)
    regexp = re.compile(r'RQ.*\?')
    matches = regexp.findall(text)
    if matches:
        print(matches)



{'section': 'INTRODUCTION AND BACKGROUND', 'text': 'Our focus is on these live and distributed lectures. In particular, we know that it can be quite problematic for the presenter to be aware of and naturally interact with remote students who are viewing the lecture as a webcast [9] . When the number of participants is small, this can sometimes be overcome using videoconferencing. As the class grows, however, it becomes more difficult [15] . Problems arise because screen space for visible representations of remote participants is limited, and humans have limited capacity for processing such representations [11] . Thus, the rich information that instructors ordinarily use to find appropriate interaction opportunities, to find good "target" students, and to time these interactions is missing. Clearly it is not possible to replicate all of the real-world cues in a distributed environment. This raises the question of whether there are critical bits of awareness information that would be use

In [80]:
def get_introduction_text(dPaper):
    texts = []
    for dSec in dPaper['body_text']:
        text = dSec['section'].lower()
        regexp = re.compile(r'intro')
        if regexp.search(text):
            texts.append(dSec['text'])
    return texts

def get_relatedWork_text(dPaper):
    texts = []
    for dSec in dPaper['body_text']:
        text = dSec['section'].lower()
        regexp = re.compile(r'related work｜relatedwork')
        if regexp.search(text):
            texts.append(dSec['text'])
    return texts

def get_RQ_text(dPaper):
    texts = []
    for dSec in dPaper['body_text']:
        text = dSec["section"] + dSec["text"]
        # non greedy match
        # regexp = re.compile(r'[- a-z([]*?(?:\d.|:) (?:what|how|why|is|are|can|to what extent) [^[?]*\?')
        regexp = re.compile(r'RQ1|RQ2|RQ3|\(RQ\)|research question')
        # regexp = re.compile(r'(?:what|how|why|is|are|can|to what extent) [^[?]*\?')
        matches = regexp.findall(text)
        if matches:
            texts.append(text)
    return texts

In [53]:
# "1. How do students assess the contribution of the flipped-classroom approach to the learning process and the watching of videos between classes as against the watching of videos in class? 
# 2. What are the relations between the assessment of the contribution of the flippedclassroom approach to the learning process and the students' background characteristics, feelings about having the lecturer and classmates nearby, and self-assessment of the learning ability?"

In [54]:
regexp = re.compile(r'RQ1|\(RQ\)|research question')
text = str({"section": "RQ1. Can DDL help EFL learners improve their lexico-grammatical use of abstract nouns in their writing?", "text": "The control group and the experimental group had similar error-free ratios in terms of the use of the ten target nouns in the pre-test, but in the post-test, the experimental group had much fewer errors in their texts. Also, even though the experimental group used various patterns in the post test, the patterns used by the control group were limited. For instance, excitement was used in a greater variety of grammatical patterns by the experimental group than by the control group. The experimental group used five grammatical patterns of excitement: 1) PP + excitement; 2) V + excitement; 3) excitement + V; 4) excitement + copular verb BE and 5) copular verb BE + excitement. However, the control group used the noun in only three patterns: 1) excitement + copular verb BE; 2) PP + excitement; 3) V + excitement. Moreover, while the control group preferred using simple patterns in the post test, the experimental group used more complex patterns. For example, the noun silence was used with various adjectives by the experimental group (sudden silence, awkward silence, strange silence, uncomfortable silence, breathless silence, unknown silence). That is, the results indicated that to a large extent, the experimental group outperformed the control group in both areas of syntactic variations and correct grammar use because of the use of DDL activities as in the studies of Huang (2014) , , Ucar and Yükselir (2015) , Vyatkina (2016) , and Yunus and Awab (2014).", "cite_spans": [{"start": 1383, "end": 1395, "text": "Huang (2014)", "ref_id": "BIBREF5"}, {"start": 1400, "end": 1424, "text": "Ucar and Yükselir (2015)", "ref_id": "BIBREF17"}, {"start": 1427, "end": 1442, "text": "Vyatkina (2016)", "ref_id": "BIBREF19"}]})
matches = regexp.findall(text)
matches

['RQ1']

In [67]:
rqs = []
intros = []
relatedWorks = []

for i in tqdm(range(100)):
    path = './s2orc_hci/pdf_parses/pdf_parses_%d.jsonl'%i

    # load jsonl file
    with open(path, 'r', encoding='utf8') as f:
        data = f.readlines()
        data = [json.loads(line) for line in data]


    for dPaper in data:
        rqs.append(get_RQ_text(dPaper))
        intros.append(get_introduction_text(dPaper))
        relatedWorks.append(get_relatedWork_text(dPaper))

100%|██████████| 100/100 [00:26<00:00,  3.75it/s]


In [68]:
len(rqs)

27955

In [69]:
len([rq for rq in relatedWorks if rq])

4034

In [70]:
[rq for rq in rqs if rq]

[["IntroductionEmpirical methods are critical to gauge the scalability and robustness of proposed approaches, to assess progress and to stimulate new research questions. In the field of natural language generation, empirical evaluation has only recently become a top research priority (Dale, Eugenio et al. 1998) . Some empirical work has been done to evaluate models for generating descriptions of objects and processes from a knowledge base (Lester and Porter March 1997) , text summaries of quantitative data (Robin and McKeown 1996) , descriptions of plans (Young to appear) and concise causal arguments (McConachy, Korb et al. 1998) . However, little attention has been paid to the evaluation of systems generating evaluative arguments, communicative acts that attempt to affect the addressee's attitudes (i.e. evaluative tendencies typically phrased in terms of like and dislike or favor and disfavor). The ability to generate evaluative arguments is critical in an increasing number of online 

In [73]:
# write results to jsonl file
# with open('allQuestions_s2orc.jsonl', 'w', encoding='utf8') as f:
with open('RQs_s2orc.jsonl', 'w', encoding='utf8') as f:
    for idx, rq in enumerate(rqs):
        if rq:
            f.write(json.dumps(
                {
                    'intro': intros[idx],
                    'relatedWork': relatedWorks[idx],
                    'rq': rq
                }
            ))
            f.write('\n')

In [76]:
# check how many papers have intro, related work and RQs
with open('RQs_s2orc.jsonl', 'r', encoding='utf8') as f:
    data = f.readlines()
    data = [json.loads(line) for line in data]
    cnt = 0
    for d in data:
        if ( d['relatedWork']) and d['rq']:
            cnt += 1
    print(cnt)

28


In [75]:
# check how many papers have intro, related work and RQs
with open('allQuestions_s2orc.jsonl', 'r', encoding='utf8') as f:
    data = f.readlines()
    data = [json.loads(line) for line in data]
    cnt = 0
    for d in data:
        if (d['intro'] and d['relatedWork']) and d['rq']:
            cnt += 1
    print(cnt)

197


In [81]:
# only extract related works
rqs = []
intros = []
relatedWorks = []

for i in tqdm(range(100)):
    path = './s2orc_hci/pdf_parses/pdf_parses_%d.jsonl'%i

    # load jsonl file
    with open(path, 'r', encoding='utf8') as f:
        data = f.readlines()
        data = [json.loads(line) for line in data]


    for dPaper in data:
        rqs.append(get_RQ_text(dPaper))
        intros.append(get_introduction_text(dPaper))
        relatedWorks.append(get_relatedWork_text(dPaper))

100%|██████████| 100/100 [00:23<00:00,  4.30it/s]


In [79]:
len([r for r in relatedWorks if r])

4034